In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

API key looks good so far


In [3]:
def get_prompts_for_company_related_urls(company_name,url):
    system_prompt = f"""
You are a link filtering assistant that evaluates search results to identify the most relevant links for a stock news summary about {company_name}.

Your task is to select atmost 10 links that are most relevant for creating a comprehensive stock news summary. Prioritize links related to:
- Company stock news and price movements
- Company financial reports, earnings, and financial performance
- Company historical data and analysis
- Company announcements and press releases
- Competitor stock news and market comparisons
- Industry trends and market analysis affecting the company
- Regulatory news or market events impacting the company

Focus on the company, its competitors, and the broader financial market context.

Exclude links that are:
- Marketing or promotional content
- Advertising pages
- Terms of Service, Privacy Policy, or legal pages
- Email links or contact pages
- General news sites without specific company relevance

You should respond in JSON format as shown in this example:
"""
    format_json = {
                        "links": [
                            {"type": "company news", "url": "https://full.url/goes/here/company_news"},
                            {"type": "competitor news", "url": "https://another.full.url/competitor_company_news"}
                        ]
                    }
    system_prompt += json.dumps(format_json)

    user_prompt = f"""
Below is a list of links extracted from the website: {url}

Your task: Analyze each link and select only those that are relevant for creating a comprehensive stock news summary about {company_name}.
Limit the number of links selected to a maximum of 10.

For each relevant link, provide:
- The complete, full URL (convert relative URLs to absolute URLs by prepending the base URL if needed)
- A descriptive type classification (e.g., "company news", "earnings report", "competitor analysis", "market trend", "financial report")

Return your response in JSON format with the structure shown in the example and follow the provided guidelines.

Links (note: some may be relative URLs that need to be converted to absolute URLs):
"""
    
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)

    return system_prompt, user_prompt

In [4]:
openai = OpenAI()

def select_relevant_links(company_name,url):
    system_prompt, user_prompt = get_prompts_for_company_related_urls(company_name,url)
    response = openai.chat.completions.create(
        model='gpt-5-nano',
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [12]:
select_relevant_links("Microsoft","https://finance.yahoo.com/quote/MSFT")

{'links': []}

In [5]:
def fetch_content_from_all_relevant_links(company_name, url):
    relevant_links = select_relevant_links(company_name, url)
    result = f"## {company_name} News Links:\n"
    
    successful_links = 0
    failed_links = 0
    
    for link in relevant_links['links']:
        link_url = link["url"]
        print(f"Fetching: {link['type']} - {link_url}")
        
        content = fetch_website_contents(link_url)
        
        # Check if content indicates an error
        if content.startswith("Error:"):
            print(f"  ⚠️  {content}")
            failed_links += 1
            continue  # Skip this link
        
        result += f"\n\n### Link: {link['type']}\n"
        result += content
        successful_links += 1
        print(f"  ✓ Success")
    
    print(f"\nSummary: {successful_links} successful, {failed_links} failed")
    return result

In [ ]:
def get_prompts_for_company_news(company_name, url):
    system_prompt = f"""
You are a financial analysis assistant specializing in stock market news and company analysis.

Your task is to analyze multiple news articles about {company_name} and create a comprehensive, well-structured summary that would be valuable for prospective customers and investors.

Your summary should:
- Provide a clear overview of the company's recent financial news and developments
- Include key financial metrics, earnings reports, and performance indicators when available
- Highlight recent announcements, strategic moves, or significant company events
- Discuss competitor performance and market positioning when relevant
- Analyze industry trends and market conditions affecting the company
- Present information in a balanced, objective manner

Exclude any content related to:
- third party ads on the page
Format your response in markdown without code blocks. Use appropriate markdown formatting (headers, bullet points, bold text) to make the summary easy to read and navigate.
"""

    user_prompt = f"""
Below are the contents of selected finance news articles about {company_name} and related market information.

Please analyze these articles and create a comprehensive stock news summary about {company_name} following the guidelines provided.

Article Contents:
"""
    user_prompt += fetch_content_from_all_relevant_links(company_name,url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    
    return system_prompt, user_prompt

In [7]:
def create_news_summary(company_name, url):
    system_prompt, user_prompt = get_prompts_for_company_news(company_name, url)
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [10]:
create_news_summary("Apple","https://finance.yahoo.com/")

Fetching: price data - https://finance.yahoo.com/quote/AAPL/
  ✓ Success
Fetching: earnings report - https://finance.yahoo.com/news/apple-tops-q1-earnings-estimates-on-record-breaking-iphone-sales-213019517.html
  ✓ Success
Fetching: market news - live coverage - https://finance.yahoo.com/news/live/stock-market-today-dow-sp-500-nasdaq-futures-slide-as-trumps-fed-pick-apple-earnings-take-focus-234128194.html
  ✓ Success
Fetching: competitor stock news - https://finance.yahoo.com/quote/MSFT/
  ✓ Success
Fetching: competitor stock news - https://finance.yahoo.com/quote/NVDA/
  ✓ Success
Fetching: market index - https://finance.yahoo.com/quote/%5EDJI/
  ✓ Success
Fetching: market data - ETF - https://finance.yahoo.com/quote/SPY/
  ✓ Success
Fetching: industry trend - https://finance.yahoo.com/sectors/technology/
  ✓ Success
Fetching: analysis video - https://finance.yahoo.com/video/ai-is-an-existential-threat-for-apple-analyst-says-221101632.html
  ✓ Success
Fetching: earnings calendar - h

# Apple Inc. (AAPL) - Financial News & Market Analysis Summary

## Overview
Apple Inc. remains one of the most closely watched technology giants in the stock market, with investors and customers eager to assess its financial performance, strategic moves, and market positioning. Recent news and data sources provide a broad view of Apple’s current standing, though some reports are limited or interrupted in content delivery. Here is a structured summary of Apple's recent financial and market situation based on available information.

---

## Recent Financial Performance & Key Metrics

- **Stock Performance**: Apple's stock continues to be actively traded with significant attention on its price movements, trading volumes, and historical performance trends. While specific up-to-date price data was temporarily inaccessible, Apple traditionally maintains a strong market capitalization and liquidity.
- **Earnings Reports**: Detailed earnings data was not explicitly available in the current news extracts, but Apple’s quarterly reports typically highlight robust revenue generation driven by iPhone sales, growing services revenue, and wearables.
- **Dividend & Shareholder Returns**: Apple has a history of returning value to shareholders through dividends and stock buybacks. Current dividend yields and payout ratios typically attract income-focused investors.

---

## Strategic Announcements and Company Developments

- **Product Innovations**: While no direct announcements surfaced in the extracted content, Apple’s ongoing product development pipeline, including iPhones, iPads, Macs, and expanding services like Apple Music and iCloud, usually drive growth.
- **Market Expansion**: Apple regularly explores new markets and product categories, potentially including augmented reality (AR), artificial intelligence (AI), and health-related technologies, aligning with broader tech industry trends.
- **Supply Chain and Manufacturing**: Apple often adjusts its supply chain strategies in response to global chip shortages, geopolitical tensions, and market demand fluctuations, impacting product availability and costs.

---

## Competitive Landscape & Market Position

- **Competitors**: Apple operates in a competitive environment including companies like Microsoft, Google (Alphabet), Amazon, and Samsung across hardware, software, and services.
- **Market Share**: Apple dominates in several segments such as premium smartphones and tablets. Its brand loyalty and ecosystem integration present competitive advantages.
- **Stock Comparisons**: Investor interest often compares Apple with peers such as Microsoft (MSFT), Nvidia (NVDA), and others to evaluate growth potential and valuation.

---

## Industry Trends & Market Conditions

- **Technology Sector Trends**: The tech sector is influenced by rapid innovation cycles, regulatory scrutiny, and shifts in consumer behavior. Apple’s ability to innovate and capture consumer interest remains crucial.
- **Macroeconomic Factors**: Interest rates, inflation, and global economic conditions affect consumer spending on discretionary tech products, influencing Apple’s sales.
- **Supply Chain Dynamics**: Ongoing disruptions or improvements in semiconductor manufacturing and logistics can affect Apple's product launch timelines and cost base.

---

## Balanced Outlook

- **Strengths**: Strong brand, diversified revenue streams, solid balance sheet, loyal customer base.
- **Challenges**: Intense competition, regulatory pressures, supply chain uncertainties.
- **Investor Considerations**: Apple presents a blend of stable long-term growth prospects underpinned by innovation and ecosystem lock-in, alongside risks from market volatility and sector-specific challenges.

---

## Conclusion

Apple Inc. remains a cornerstone of the technology sector with substantial influence on global markets. Despite the temporary inaccessibility of detailed current price and news data, the company’s historical performance, strategic initiatives, and strong competitive positioning continue to make it a focal point for investors. Prospective shareholders should monitor upcoming earnings reports, product announcements, and macroeconomic factors to gauge future performance.

---

*Note: This summary excludes advertisement content and focuses solely on relevant financial and market information about Apple Inc.*